# Heuristic Code

In [50]:
import numpy as np
def esp(x, k):
    n = len(x)
    if k == 0:
        return 1
    if k == 1:
        return sum(x)
    S = np.zeros((n+1, k))
    for j in range(1, n+1):
        S[j, 0] = S[j-1, 0] + x[j-1]
    for i in range(1, k):
        for j in range(1, n+1):
            S[j, i] = S[j-1, i] + x[j-1] * S[j-1, i-1]
    return S[n, k-1]

def char_coeff_eigen(X, k):
    return esp(np.linalg.eigvalsh(X), k)

def char_coeff(X, k):
    if k == 0:
        return 1
    #if k < 8:
    #return char_coeff_FL(X, k)
    return char_coeff_eigen(X, k)
    #return char_poly_built(X, k)
    
def maximal_root(p):
    return max(np.roots(p))

def newton_method(p):
    # start at a point that is larger than the maximum root of p
    x = 1 + max([abs(c/p[0]) for c in p[1:]])
    d = len(p)
    dpdx = [(d - i) * p[i] for i in range(d-1)]
    
    iters = 0
    while abs(evaluate(p, x)) > 1e-3:
        x -= evaluate(p, x) / evaluate(dpdx, x)
        iters += 1
        if iters > 1000:
            print("failed")
            break
    return x

def evaluate(p, x):
    # Horn evaluation
    val = 0
    for c in p:
        val *= x
        val += c
    return val

def swap(X, i, j):
    if i == j:
        return
    for k in range(len(X)):
        X[k,i], X[k,j]  = X[k,j], X[k,i]
    for k in range(len(X)):
        X[i,k], X[j,k]  = X[j,k], X[i,k]
        
def conditional_char(X, t, k):
    schur = X[t:, t:] - X[t:, :t] @ np.linalg.inv(X[:t, :t]) @ X[:t, t:]
    return np.linalg.det(X[:t, :t]) * char_coeff(schur, k-t)

def root_heur(X, t, k, D = None):
    if D is None:
        D = np.eye(len(X))
    # Evaluate p(X + t I) in k places
    vals = [conditional_char(i * 0.01*np.eye(len(X)) - X, t+1, k) for i in range(k + 1)]
    # Find the coefficients of p(X+tI)
    p = np.polyfit([0.01 * x for x in range(k + 1)], vals, k)
    
    # Use Newtons' method to find maximal root
    return newton_method(p)

def find_subset(A, k):
    n = A.shape[1]
    T = []
    X = A.copy()
    #Normalize X so that its eigenvalues lie in the range [1/2, 1]
    eigs = np.linalg.eigvalsh(X)
    max_eig = max(eigs)
    min_eig = min(eigs)
    scal = 1/(2*(max_eig - min_eig))
    X = scal * X + (0.5 - min_eig * scal) * np.eye(n)
    
    bests = []
    for t in range(k):
        best = -1
        best_heur = 0
        for j in range(t, n):
            swap(X, t, j)
            heur = root_heur(X, t, k)

            if heur > best_heur:
                best = j
                best_heur = heur
            swap(X, t, j)
        swap(X, t, best)
        try:
            while True:
                best = T.index(best)
        except ValueError:
            T.append(best)
        bests.append(best_heur - 1)
    return T, bests

In [51]:
from sklearn.datasets import load_wine
import numpy as np
data = load_wine()
A = (np.transpose(data.data) @ data.data)

In [52]:
find_subset(A, 5)

([12, 4, 3, 0, 9],
 [-0.004673254020837292,
  -0.0001930465295686412,
  0.0002082560271872147,
  0.0012723296744916457,
  0.010349967878908739])

In [39]:
l = [12, 4, 3, 0, 9]
np.linalg.eigvalsh([[A[i,j] for i in l] for j in l])

array([2.80162563e+02, 8.16717844e+02, 3.20754182e+03, 2.43073296e+05,
       1.18514598e+08])

In [56]:
import random
max_eig = 0
best = []
for i in range(13):
    for j in range(i+1, 13):
        for k in range(j+1, 13):
            for g in range(k+1, 13):
                for h in range(g+1, 13):
                    l = [i,j,k,g,h]
                    m = max(np.linalg.eigvalsh([[A[i,j] for i in l] for j in l]))
                    if m > max_eig:
                        max_eig = m
                        best = l
                    
print(max_eig)
print(l)

118514597.50361837
[8, 9, 10, 11, 12]


In [30]:
conditional_char(np.diag([3,1,3]), 0, 1)

3.0000000000000004

In [72]:
newton_method(esp(list(range(6)), 5))

275.0
272.7292558639588
270.4774353452898
268.2443807534807
266.02993571216
263.833945148146
261.6562552805872
259.4967136101936
257.35516890855735
255.23147120756252
253.12547178888317
251.03702317356888
248.9659791117171
246.91219457223156
244.875525732666
242.85582996915267
240.8529658464145
238.8667931078608
236.89717266576534
234.94396659152636
233.00703810600754
231.08625156995984
229.1814724745228
227.2925674318052
225.4194041655442
223.56185150184228
221.71977935998137
219.89305874331362
218.08156173022792
216.28516146519186
214.50373214986834
212.7371490343061
210.98528840820373
209.24802759224647
207.5252449295153
205.8168197769675
204.1226324969882
202.4425644490125
200.77649798121726
199.12431642228216
197.48590407321947
195.86114619927187
194.2499290218778
192.65213971070375
191.06766637574304
189.49639805948036
187.9382247291216
186.3930372688885
184.86072747237748
183.3411880349822
181.8343125463792
180.3399954830762
178.85813220102247
177.3886189282809
175.9313527577609

5.000000414815706

In [49]:
print(evaluate(esp([5,4,3,2],4),4.95))

0.27324375000020495


### find_subset(A, b, 5)

In [104]:
T

[1, 4, 58, 74, 91]

In [98]:
np.linalg.norm(A[:, 0])

5.720431977587881

In [100]:
np.linalg.norm(A[:, 47])

6.697812512690744

## Orthogonal Matching Pursuit

In [77]:
def omp(A, b, k):
    n = A.shape[1]
    A = A.copy()
    b = b.copy()
    T = []
    bests = []
    for t in range(k):
        best = -1
        best_obj = 0
        
        for i in range(n):
            if i in T:
                continue
            obj = np.dot(b, A[:,i]) / np.linalg.norm(A[:,i])
            if obj > best_obj:
                best = i
                best_obj = obj 
        T.append(best)
        best_vec = A[:, best].copy() / np.linalg.norm(A[:,best])
        for i in range(n):
            A[:,i] -= np.dot(best_vec, A[:,i]) * best_vec
        b -= np.dot(best_vec, b) * best_vec
        bests.append(np.linalg.norm(b)**2)
    return T, bests

# Regression with random matrices

In [195]:
from sklearn import linear_model
clf = linear_model.Lasso(alpha=0.02)
import random
def test(n,m,k):
    T = random.sample(list(range(n)), k)
    T.sort()
    print("T: ", T)
    A = np.random.normal(loc = 0, scale = 1, size = (m,n))
    b = sum(A[:,i] for i in T)
    S1, scores1 = find_subset(A, b, k)
    print("characteristic: ", S1)
    S1.sort()
    s1 = all(s == t for t, s in zip(T, S1))
    lasso = clf.fit(A, b).coef_
    S2 = [a[1] for a in sorted([(-abs(x),i) for i,x in enumerate(lasso)])[:k]]
    S2.sort()
    s2 = all(s == t for t, s in zip(T, S2))
    S3, scores3 = omp(A,b,k)
    print("omp: ", S3)
    S3.sort()
    s3 = all(s == t for t, s in zip(T, S3))
    norm_b = np.linalg.norm(b)
    print([(norm_b**2 - x1, x3) for x1, x3 in zip(scores1, scores3)])
    return (s1,s2,s3, all(norm_b**2 - x1 >= x3 for x1, x3 in zip(scores1, scores3)))

In [196]:
n = 100
m = 30
iters = 10
count_dict = []
for k in range(6,25):
    counts = [0,0,0]
    for i in range(iters):
        for i, result in enumerate(test(n,m,k)):
            if result:
                counts[i] += 1
    print(counts)
    count_dict.append(counts)
print(count_dict)

T:  [6, 59, 81, 90, 93, 98]
characteristic:  [98, 90, 81, 59, 93, 6]
omp:  [98, 90, 81, 59, 93, 6]
[(78.5567476546986, 96.15139799916021), (42.19170330247985, 51.0749711484665), (26.298345067500534, 30.109910306477303), (16.75938889672969, 17.95774474859308), (10.84151182312371, 11.349514483140174), (6.110667527536862e-12, 5.3260148164570724e-30)]
T:  [22, 28, 29, 35, 69, 84]
characteristic:  [84, 29, 28, 69, 35, 22]
omp:  [84, 29, 28, 69, 35, 22]
[(86.64077023449, 106.87510435196278), (61.19140295013342, 72.43742640020236), (34.97751963082112, 39.906581500965146), (25.25554681030829, 27.445135404494597), (13.982029027319584, 14.703309871171538), (-9.379164112033322e-13, 7.694983879691824e-30)]
T:  [4, 6, 8, 11, 31, 87]
characteristic:  [4, 8, 31, 6, 87, 11]
omp:  [4, 8, 31, 6, 87, 11]
[(84.84673991597026, 105.07578723960935), (61.03025655436997, 72.58713101876899), (46.96144568929046, 53.54635997550257), (32.42299197713035, 35.30114389970806), (16.281167185737388, 17.02912646446662), 

characteristic:  [85, 52, 62, 31, 59, 34, 0, 45]
omp:  [85, 52, 62, 31, 59, 34, 77, 53]
[(68.3843563017091, 89.60040160974084), (58.46961715454255, 74.84641762131567), (47.41196380818835, 57.59020340942249), (36.08831609527488, 41.94479386510055), (27.26294317791954, 30.27585308351808), (21.59419131750559, 23.314039448777756), (17.39811047750473, 18.268816949644588), (14.953435348562152, 14.88675850505076)]
T:  [26, 39, 45, 46, 47, 63, 65, 85]


KeyboardInterrupt: 

In [148]:
n = 10
m = 4
k = 2
import random
for _ in range(1000):
    T = random.sample(list(range(n)), k)
    T.sort()
    A = np.random.normal(loc = 0, scale = 1, size = (m,n))
    b = sum(A[:,i] for i in T)
    S1, scores1 = find_subset(A, b, k)
    S1.sort()
    s1 = all(s == t for t, s in zip(T, S1))
    
    S3, scores3 = omp(A,b,k)
    S3.sort()
    s3 = all(s == t for t, s in zip(T, S3))
    norm_b = np.linalg.norm(b)
    
    norm_b = np.linalg.norm(b)
    print([(norm_b**2 - x1, x3) for x1, x3 in zip(scores1, scores3)])
    
    if s3 and not s1:
        print(A)
        print(b)
        print(T)
        print(find_subset(A, b, k)[0])
        print(omp(A,b,k)[0])
        break

[(1.0377406341161688, 1.7883167077026783), (-1.0658141036401503e-14, 3.158525108795067e-32)]
[(0.24229604056866982, 2.4289620176113815), (0.047786817478977284, 0.13481097900135497)]
[(0.31727381390459364, 0.41440234987234187), (-3.2507330161024584e-13, 7.395570986446985e-32)]
[(1.339812540330998, 2.440332914421233), (0.24086240215732246, 0.4553118440461411)]
[(0.28340747903018304, 0.5015380398748129), (-2.842170943040401e-14, 2.5422275265911513e-32)]
[(0.664353795170614, 1.0460361559754618), (-2.6645352591003757e-15, 4.00593428432545e-32)]
[(0.5922546708244374, 1.0826651607376028), (-5.329070518200751e-15, 2.6500796034768365e-31)]
[(4.2263431721187885, 6.704083527907993), (-7.105427357601002e-15, 9.86076131526265e-32)]
[(0.904208533410241, 1.9984670163133653), (7.105427357601002e-15, 4.005934284325451e-31)]
[(3.0584193313259167, 4.756941524308665), (0.16625928201482232, 0.7726115466844681)]
[(0.9510678296333079, 1.4972922969486562), (0.0, 9.88002061470652e-32)]
[(0.07434752997253824, 0

In [149]:
np.set_printoptions(precision=2)
print(A)
print(T)
print(b)
B = A.copy()
for i in range(B.shape[1]):
    B[:, i] /= np.linalg.norm(B[:, i])
print(B)
print(find_subset(B, b, 2))

[[ 0.03 -0.32  0.3  -0.3  -0.85  2.19  0.48  0.55  0.8   0.38]
 [-1.12 -0.57 -0.04  0.17 -0.84 -2.53 -0.25  0.56 -0.74  0.54]
 [-0.08 -1.47  0.83  0.48  0.26  0.08 -1.63  0.64  0.09  0.2 ]
 [ 0.66 -0.73 -1.47 -0.86 -0.73  0.93  0.9  -0.85 -0.17  0.23]]
[0, 6]
[ 0.5  -1.37 -1.71  1.56]
[[ 0.02 -0.18  0.17 -0.29 -0.6   0.63  0.25  0.42  0.72  0.52]
 [-0.86 -0.32 -0.02  0.16 -0.59 -0.73 -0.13  0.43 -0.67  0.75]
 [-0.06 -0.83  0.49  0.46  0.18  0.02 -0.84  0.48  0.08  0.27]
 [ 0.51 -0.41 -0.86 -0.82 -0.51  0.27  0.46 -0.64 -0.16  0.32]]
([3, 9], [6.656037691903282, 7.272780916696597])


In [165]:
print(omp(B, b, 2))

([6, 0], [0.1906553480939401, 1.0977800683007244e-32])


In [156]:
X = np.transpose(B) @ B
for i in range(10):
    for j in range(i+1, 10):
        print("T ", [i, j])
        A = [[X[i,i], X[i,j]],[X[i,j], X[j,j]]]
        print(np.linalg.det(A))
        A = [[Z[i,i], Z[i,j]],[X[i,j], X[j,j]]]

T  [0, 1]
0.9869531513931562
T  [0, 2]
0.80569147504437
T  [0, 3]
0.6545237267574593
T  [0, 4]
0.9512010688050985
T  [0, 5]
0.4015654674204535
T  [0, 6]
0.8383549957583887
T  [0, 7]
0.4928648660746797
T  [0, 8]
0.742713691867894
T  [0, 9]
0.7635222447573202
T  [1, 2]
0.9938552644502497
T  [1, 3]
0.9980747923025646
T  [1, 4]
0.8734190003087954
T  [1, 5]
0.9999077376044934
T  [1, 6]
0.7424188124532818
T  [1, 7]
0.8792268955989423
T  [1, 8]
0.9938253203486659
T  [1, 9]
0.5223869602772246
T  [2, 3]
0.2321382144817874
T  [2, 4]
0.8070720299961356
T  [2, 5]
0.9914545790702185
T  [2, 6]
0.42049528777323164
T  [2, 7]
0.27972103536305437
T  [2, 8]
0.9021185377899353
T  [2, 9]
0.9956379679656466
T  [3, 4]
0.6545768668909977
T  [3, 5]
0.7430075239820985
T  [3, 6]
0.2580157168516969
T  [3, 7]
0.5095363660206739
T  [3, 8]
0.9780665693878657
T  [3, 9]
0.9719513878337964
T  [4, 5]
0.9934207914404204
T  [4, 6]
0.7850377783855316
T  [4, 7]
0.9939908708925076
T  [4, 8]
0.9965888824078386
T  [4, 9]
0.258

In [179]:
X = np.transpose(B) @ B
b /= np.linalg.norm(b)
D_tot = np.eye(10)
for t in range(10):
    for i in range(10):
        swap(X, 0, i)
        swap(D_tot, 0, i)
        D = np.eye(10)
        D[0,0] = 1/np.sqrt(conditional_char(X, 1, 2))
        X = D @ X @ D
        D_tot = D @ D_tot
Z = X + D_tot @ np.transpose(B) @ np.outer(b, b) @ B @ D_tot
for i in range(10): 
    print(i)
    swap(X, 0, i)
    swap(Z, 0, i)
    print(conditional_char(X, 1, 2))
    print(conditional_char(Z, 1, 2))

0
0.9999999999999996
2.854683636313858
1
0.999994825514956
2.5630020695310787
2
0.9999974496609838
2.8259436624628367
3
0.9999986712071594
2.9907139761711306
4
0.9999989880040638
2.497547943582923
5
0.9999997363333737
2.7805889430265482
6
1.0000018321009967
3.1150202380092353
7
1.000001470413924
2.86550964971077
8
1.0000010863137998
2.513131211551526
9
1.0000006216914992
2.524655654270109


In [73]:
A = np.random.normal(loc = 0, scale = 1, size = (25,400))

In [140]:
B

array([[ 0.48, -0.63,  0.54,  0.57,  0.94, -0.06, -0.44,  0.18,  0.21,
        -0.23],
       [-0.57,  0.65,  0.13,  0.01,  0.05, -0.45,  0.42,  0.02,  0.18,
         0.89],
       [ 0.57,  0.43, -0.58, -0.73,  0.18, -0.87, -0.77,  0.74, -0.89,
         0.13],
       [ 0.36, -0.01,  0.59, -0.38,  0.3 , -0.22,  0.2 ,  0.64,  0.37,
        -0.36]])

In [81]:
# An example of sparse regression.
print(find_subset(A, A[:,1]+A[:,3]-A[:,5], 3))

([3, 1, 5], [52.8597368095047, 80.41534373321, 92.79035322897276])


In [75]:
from sklearn import linear_model
clf = linear_model.Lasso(alpha=0.02)
clf.fit(A, A[:,1]+A[:,3]-A[:,5])
lasso = (clf.coef_)
print(lasso)
[a[1] for a in sorted([(-abs(x),i) for i,x in enumerate(lasso)])[:3]]

[ 0.          0.98631157 -0.          0.98396293 -0.         -0.98040089
  0.         -0.         -0.         -0.         -0.          0.
  0.         -0.          0.          0.          0.          0.
 -0.          0.          0.          0.         -0.          0.
  0.         -0.         -0.         -0.         -0.          0.
  0.          0.          0.          0.         -0.         -0.
  0.         -0.         -0.          0.          0.          0.
  0.         -0.         -0.         -0.          0.          0.
 -0.         -0.         -0.          0.          0.          0.
  0.          0.         -0.          0.         -0.         -0.
  0.         -0.         -0.         -0.         -0.         -0.
  0.          0.          0.         -0.         -0.         -0.
 -0.          0.         -0.          0.         -0.         -0.
 -0.         -0.          0.          0.         -0.          0.
  0.         -0.          0.         -0.          0.         -0.
 -0.         -0. 

[1, 3, 5]

In [78]:
print(omp(A, A[:,1]+A[:,3]-A[:,5], 3))

([3, 1, 22], [43.361546022585614, 12.859484606413732, 9.779391183513335])


# Pandas Dataframe Example

In [202]:
import pandas as pd
df = pd.read_csv("winequality-red.csv", delimiter=";")
A = df.to_numpy()

In [203]:
b = A[:, 11]
A = A[:, :11]
print(find_subset(A, b,4))

([10, 7, 1, 9], [51030.09221174199, 51100.599504954225, 51126.869164146425, 51142.02405921621])


In [206]:
print(omp(A, b,4))

([10, 9, 0, 7], [863.9888540225741, 789.5176091327127, 757.0546707781789, 747.0084094386499])


In [204]:
# volatile aacidity, density, sulphates, alcohol
lin_reg(A[:,[10, 9, 7, 1]], b)

691.9758795394227

In [207]:
# volatile aacidity, density, sulphates, alcohol
lin_reg(A[:,[10, 9, 7, 0]], b)

747.0084094394042

In [208]:
from sklearn import linear_model
clf = linear_model.Lasso(alpha=0.1)
clf.fit(A, b)
print(clf.coef_)

[ 0.03 -0.    0.    0.   -0.    0.01 -0.   -0.   -0.    0.    0.26]


In [101]:
# fixed acidity, free sulfur dioxide, total sulphur dioxide, alcohol.
lin_reg(A[:,[0,5,6,10]],b)

793.3826151041285

In [102]:
df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


# Sklearn Diabetes Data

In [210]:
from sklearn.datasets import load_diabetes
diabetes = load_diabetes()

In [211]:
find_subset(diabetes["data"], diabetes["target"], 4)

([2, 8, 3, 4],
 [1115549.036535078,
  1239525.3985064633,
  1272369.0609363422,
  1289577.7224586348])

In [212]:
lin_reg(diabetes["data"][:,[2, 8, 3, 4]], diabetes["target"])

11561343.279130083

In [213]:
omp(diabetes["data"], diabetes["target"], 4)

([2, 8, 3, 7],
 [11949493.686339522,
  11646605.889522208,
  11592620.569271391,
  11589401.89591285])

In [214]:
lin_reg(diabetes["data"][:,[2, 8, 3, 7]], diabetes["target"])

11589401.895912847

In [215]:
clf = linear_model.Lasso(alpha=0.5)
clf.fit(diabetes["data"], diabetes["target"])
print(clf.coef_)

[  0.    -0.   471.04 136.5   -0.    -0.   -58.32   0.   408.02   0.  ]


In [107]:
lin_reg(diabetes["data"][:, [2, 3, 6, 8]], diabetes["target"])

11562699.344660645

# Small inner products

In [69]:
k = 4
m = 2 * k
for i in range(10000):
    true_dot = 1/(2*k-1) * (2*np.random.rand()-1)
    true_bad_dot =  1/(2*k-1) * (2*np.random.rand()-1)
    bad_dot =  1/(2*k-1) * (2*np.random.rand()-1)

    A = np.array([[1 if i == j else (true_dot if i < k and j < k else (bad_dot if i >= k and j >= k else true_bad_dot)) for i in range(m)] for j in range(m)])
    b = A @ [1 if i < k else 0 for i in range(m)]
    Z = A + np.outer(b,b)
    a0 = conditional_char(Z, 1, k)/conditional_char(A, 1, k)
    swap(Z, 0, k)
    a1 = conditional_char(Z, 1, k)/conditional_char(A, 1, k)
    swap(Z, 0, k)
    if a0 < a1:
        print("failed")
        break

In [64]:
print(conditional_char(Z, 1, k))
print(conditional_char(A, 1, k))
print(conditional_char(Z, 1, k)/conditional_char(A, 1, k))
swap(Z, 0, k)
print(conditional_char(Z, 1, k))
print(conditional_char(A, 1, k))
print(conditional_char(Z, 1, k)/conditional_char(A, 1, k))
swap(Z, 0, k)

8.184996409600839
2.799782826643721
2.923439751008373
6.341385266118527
2.799782826643721
2.264956126515124


In [45]:
np.linalg.eigvalsh(A)

array([0.15, 1.  , 1.33, 1.52])

In [46]:
np.set_printoptions(precision=2)
A

array([[ 1.  , -0.33, -0.33, -0.33],
       [-0.33,  1.  , -0.33, -0.33],
       [-0.33, -0.33,  1.  ,  0.  ],
       [-0.33, -0.33,  0.  ,  1.  ]])